In [1]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using ca

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
nawon = pd.read_excel("1_내원정보.xlsx")
nawon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249619 entries, 0 to 249618
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   VTHOS_NO       249619 non-null  object 
 1   RGNO           249619 non-null  object 
 2   AGRDE          249619 non-null  object 
 3   SEX_CD         249619 non-null  object 
 4   AREA_ADRES     247923 non-null  object 
 5   VTHOS_CLSF_CD  249619 non-null  object 
 6   IADM_DEPTT_NM  249619 non-null  object 
 7   DEPTT_NM       249619 non-null  object 
 8   ADM_YMD        249619 non-null  int64  
 9   LTHT_YMD       249619 non-null  int64  
 10  ESB_TX_ID      0 non-null       float64
 11  ESB_STATE_CD   0 non-null       float64
 12  ESB_TIME       0 non-null       float64
dtypes: float64(3), int64(2), object(8)
memory usage: 24.8+ MB


In [3]:
nawon.rename(columns={'VTHOS_NO': '내원번호', 'RGNO': '환자번호',
                     'AGRDE':'환자연령대','SEX_CD':'환자성별','AREA_ADRES':'환자거주지역',
                     'VTHOS_CLSF_CD':'외래/응급여부','IADM_DEPTT_NM':'진료부서',
                     'ADM_YMD':'입원일자','LTHT_YMD':'퇴원일자'}, inplace=True)
nawon.drop(['ESB_TX_ID', 'ESB_STATE_CD', 'ESB_TIME'], axis=1 ,inplace=True)
nawon['입원일자'] = nawon['입원일자'].map(lambda x : 20000000+(x//1000000)*10000+(x%10000))
nawon['퇴원일자'] = nawon['퇴원일자'].map(lambda x : 20000000+(x//1000000)*10000+(x%10000))

In [4]:
nawon['입원일자'] = pd.to_datetime(nawon['입원일자'], format='%Y%m%d', errors='coerce')
nawon['퇴원일자'] = pd.to_datetime(nawon['퇴원일자'], format='%Y%m%d', errors='coerce')
# 윤년버그로 인해서 2021-02-29는 존재하지 않는다. 2021-02-28로 전부 대체한다.
nawon['입원일자'].fillna(pd.to_datetime('20210228', format='%Y%m%d'), inplace=True)
nawon['퇴원일자'].fillna(pd.to_datetime('20210228', format='%Y%m%d'), inplace=True)
nawon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249619 entries, 0 to 249618
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   내원번호      249619 non-null  object        
 1   환자번호      249619 non-null  object        
 2   환자연령대     249619 non-null  object        
 3   환자성별      249619 non-null  object        
 4   환자거주지역    247923 non-null  object        
 5   외래/응급여부   249619 non-null  object        
 6   진료부서      249619 non-null  object        
 7   DEPTT_NM  249619 non-null  object        
 8   입원일자      249619 non-null  datetime64[ns]
 9   퇴원일자      249619 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(8)
memory usage: 19.0+ MB


In [5]:
vatang = pd.read_excel("2_바탕질환.xlsx")


In [6]:
vatang.drop(['RGNO','ESB_TX_ID', 'ESB_STATE_CD', 'ESB_TIME'], axis=1 ,inplace=True)

In [7]:
vatang.rename(columns={'VTHOS_NO': '내원번호','ICD10CD':'ICD10코드','DIAG_NM':'진단명',
                     'MAIN_DIAG_YN':'주진단여부'}, inplace=True)
vatang

,내원번호,ICD10코드,진단명,주진단여부
0,0xa08a0f54814fdb94e623cfe7d0cf992c6b42410388f5...,J21.9,Bronchiolitis,Y
1,0xa08a0f54814fdb94e623cfe7d0cf992c6b42410388f5...,A49.3,Mycoplasma infection,N
2,0xa08a0f54814fdb94e623cfe7d0cf992c6b42410388f5...,J18.9,Pneumonia,N
3,0x05635cc56defcb7f1772d06accb9569e9daf375025e4...,J45.9,Asthma,Y
4,0xeb2054397645832f85b81f005333ee48d557b2460d3d...,T81.0,S/P tonsillectomy bleeding,Y
...,...,...,...,...
458256,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,B18.19,"Hepatitis B, viral, chronic",N
458257,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,K81.0,Empyema of gallbladder,N
458258,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,E87.2,Acidosis,N
458259,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,Z11.5,Special screening examination for Coronavirus ...,N


In [8]:
vatang.value_counts()

내원번호                                                                ICD10코드  진단명                               주진단여부
0x3622215fe85291a48aaf2db76174aa95c453709e270fc4e0c028c79c2f9febcb  D48.0    Spine tumor (uncertain behavior)  N        3
                                                                    J38.09   Vocal cord palsy                  N        3
                                                                    N31.9    Neurogenic bladder                N        3
                                                                    M79.100  Myofascial pain syndrome          N        3
                                                                    G20      Parkinson disease                 N        3
                                                                                                                       ..
0x5549323a12bfc731945c0693a81ea5b3646e9f3dd43ed415365bb7d20462c450  G52.9    Cranial nerve disorder            N        1
                             

In [9]:
nawon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249619 entries, 0 to 249618
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   내원번호      249619 non-null  object        
 1   환자번호      249619 non-null  object        
 2   환자연령대     249619 non-null  object        
 3   환자성별      249619 non-null  object        
 4   환자거주지역    247923 non-null  object        
 5   외래/응급여부   249619 non-null  object        
 6   진료부서      249619 non-null  object        
 7   DEPTT_NM  249619 non-null  object        
 8   입원일자      249619 non-null  datetime64[ns]
 9   퇴원일자      249619 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(8)
memory usage: 19.0+ MB


In [10]:
hapbon = pd.merge(nawon,vatang,right_on='내원번호',left_on='내원번호')


In [11]:
hapbon2 = hapbon.copy()

In [12]:
ICD_dict = {
    'A': '세균/바이러스/기생 생물 감염',
    'B': '세균/바이러스/기생 생물 감염',
    'C': '악성 종양',
    'D': '양성 종양/ 면역 장애',
    'E': '내분비/대사 질병',
    'F': '정신/행동장애',
    'G': '신경 계통 질병',
    'H': '눈/귀 질병',
    'I': '순환 계통 질병',
    'J': '호흡 계통 질병',
    'K': '소화 계통 질병',
    'L': '피부/피하조직 질병',
    'M': '근육계통/연결조직 질병',
    'N': '비뇨/생식계통 질병',
    'O': '임신/출산/산욕',
    'P': '출생전후 상태',
    'Q': '기형,변형,염색체이상',
    'R': '미분류,비정상 발견',
    'S': '상해/중독',
    'T': '상해/중독',
    'U': '특별목적코드',
    'V': '질병/사망의 외부적 원인',
    'W': '질병/사망의 외부적 원인',
    'X': '질병/사망의 외부적 원인',
    'Y': '질병/사망의 외부적 원인',
    'Z': '건강 상태 영향/보건 서비스와의 관계',

}

In [13]:
hapbon2['ICD10코드'] = hapbon2['ICD10코드'].str[0].replace(ICD_dict)

In [14]:
hapbon2['진료부서'].unique()

array(['OS', 'NS', 'G4', 'G1', 'NM', 'GB', 'EY', 'GU', 'GT', 'GE', 'M1',
       'G5', 'P1', 'M3', 'GC', 'OB', 'DC', 'GN', 'GD', 'HM', 'GV', 'ETC',
       'UR', 'GX', 'GJ', 'C1', 'HO', 'GF', 'G6', 'M7', 'M5', 'G7', 'GS',
       'M8', 'GW', 'G8', 'GR', 'EN', 'GL', 'PS', 'GO', 'GZ', 'CM', 'GY',
       'G3', 'RH', 'NU', 'GG', 'EM', 'M9', 'G2', 'SK', 'GQ', 'AN', 'GA',
       'HC', 'PY', 'P7', 'GK', 'GH', 'PR'], dtype=object)

In [15]:
DEPTT_dict = {
    'GS': '일반외과',
    'NS': '신경외과',
    'CS': '흉부외과',
    'OS': '정형외과',
    'PS': '성형외과',
    'OB': '산부인과',
    'EN': '이비인후과',
    'EY': '안과',
    'UR': '비뇨기과',
    'NU': '신경과',
    'NR': '신경과',
    'NP': '신경정신과',
    'PY': '정신의학과',
    'RM': '재활의학과',
    'AN': '마취과',
    'FM': '가정의학과',
    'RD': '방사선과',
    'CP': '진단검사의학과',
    'AP': '해부병리과',
    'GI': '소화기내과',
    'ED': '내분비내과',
    'RH': '류머티스내과',
    'HO': '혈액종양내과',
    'RO': '방사선종양학과',
    'NH': '신장내과',
    'IF': '감염내과',
    'M1': '소화기내과-위장',
    'M3': '순환기내과-심혈관',
    'GB': '감염내과',
    'NM': '정신건강의학과',
    'GT': '호흡기내과(코로나포함)',
    'P1': '호흡기내과',
    'GD': '소화기내과-간',
    'M8': '소화기내과-신장',
    'HM': '건강의학과',
    'G5': '소화기내과-대장',
    'GE': '순환기내과-혈압/고지혈증',
    'CM': '내분비대사내과',
    'GN': '신경과',
    'M7': '코로나내과',
    'G6': '소화기내과-맹장,담낭',
    'GU': '비뇨의학과',
    'G7': '호흡기내과-유방질환,폐암',
    'GZ': '소화기내과-신장',
    'GW': '암센터',
    'G1': '외과',
    'GJ': '정신건강의학과',
    'C1': '순환기내과',
    'G8': '내분비대사내과-갑상선',
    'GV': '내분비대사내과-당뇨',
    'M5': '소화기내과-당뇨'
}

In [16]:
hapbon2['진료부서'] = hapbon2['진료부서'].map(lambda x: DEPTT_dict.get(x, x))

In [17]:
hapbon2['진료부서'].unique()

array(['정형외과', '신경외과', 'G4', '외과', '정신건강의학과', '감염내과', '안과', '비뇨의학과',
       '호흡기내과(코로나포함)', '순환기내과-혈압/고지혈증', '소화기내과-위장', '소화기내과-대장', '호흡기내과',
       '순환기내과-심혈관', 'GC', '산부인과', 'DC', '신경과', '소화기내과-간', '건강의학과',
       '내분비대사내과-당뇨', 'ETC', '비뇨기과', 'GX', '순환기내과', '혈액종양내과', 'GF',
       '소화기내과-맹장,담낭', '코로나내과', '소화기내과-당뇨', '호흡기내과-유방질환,폐암', '일반외과',
       '소화기내과-신장', '암센터', '내분비대사내과-갑상선', 'GR', '이비인후과', 'GL', '성형외과',
       'GO', '내분비대사내과', 'GY', 'G3', '류머티스내과', 'GG', 'EM', 'M9', 'G2',
       'SK', 'GQ', '마취과', 'GA', 'HC', '정신의학과', 'P7', 'GK', 'GH', 'PR'],
      dtype=object)

In [18]:
hapbon2['진료부서'].value_counts().head(30)

진료부서
소화기내과-위장         47686
순환기내과-심혈관        46489
정형외과             46325
정신건강의학과          27605
감염내과             23420
소화기내과-신장         19944
신경외과             19207
산부인과             18958
호흡기내과(코로나포함)     15209
호흡기내과            14757
소화기내과-간          14535
건강의학과            13203
소화기내과-대장         11867
혈액종양내과           11305
신경과               9525
순환기내과-혈압/고지혈증     8636
안과                8622
내분비대사내과           8180
코로나내과             7796
소화기내과-맹장,담낭       7769
비뇨의학과             6769
호흡기내과-유방질환,폐암     5915
일반외과              5746
암센터               5305
외과                4893
순환기내과             4438
내분비대사내과-갑상선       4197
내분비대사내과-당뇨        3930
소화기내과-당뇨          3867
비뇨기과              3737
Name: count, dtype: int64

In [25]:
hwanza = pd.DataFrame(hapbon2['환자번호'].unique(),columns=['환자'])
hwanza

,환자
0,0xb737bf38f0b08920cedff6bf8a19ff7dbbc00b741a53...
1,0x03a4abb30036750fe6170dc789e7f06c7279cd75c7a6...
2,0x66857189f4134314288f09beb96fd4b6dc68ee8452a0...
3,0x3ff79d3bb594b3e32bb606495906df2058efeb5b6e37...
4,0x1e9c9f24788c02828b272e76ce08063741953fc66a83...
...,...
68811,0xc4e490d26e2805520061a9cb587f36e86c8f42a8866d...
68812,0x59d551b9273e8a66cc0994e15f9c4007cec56d5408dd...
68813,0xa92bbad07e975b32b0bf32ebd803ff6d911702be934d...
68814,0x06771025eed31fdeade626e80a7aa4efa7ec93bc2d50...


In [39]:
for i in range(len(hwanza)):
    for j in range(len(nawon)):
        if hwanza['환자'][i] == nawon['환자번호'][j]:
            hwanza['환자성별'][i] = nawon['환자성별'][j]
            hwanza['진료부서'][i] = nawon['진료부서'][j]
            hwanza['환자거주지역'][i] = nawon['환자거주지역'][j]
            hwanza['환자연령대'][i] = nawon['환자연령대'][j]
            continue
hwanza # 너무 오래걸리는 반복문이다. 

KeyboardInterrupt: 

In [138]:
nawon3 = nawon.copy()
nawon3 = nawon3.sort_values('환자번호')
nawon3 = nawon3.reset_index().drop(['index'],axis=1)
nawon4 = pd.DataFrame(nawon3['환자번호'].value_counts())
nawon4 = nawon4.sort_index() 

In [139]:
nawon3 = nawon3.head(19)

In [140]:
nawon4 = nawon4.head(10)

In [141]:
for i in range(len(nawon4)):
    while nawon4['count'][-i-1] > 0:
        nawon3 = nawon3.drop(nawon3.index[:])
        nawon4['count'][-i-1] = 0

In [142]:
nawon4

,count
환자번호,
0x000072b4c3266952b98b006c1ce118263c02a84febad142dc30b1331759e59ce,0
0x00007445e281b90722ce72d2feabab3eb6ee61c4bef5aca51e84390629198a49,0
0x00015b36e378adaf1e926dc33b9462fd691bd69a68fdfe1819d34445f799a694,0
0x00033515c693eed93f209060c7dbdf5ea9c9f95c6d87fd0fea37ded48126cf20,0
0x000336f2a0a12e71e63624e4b3c908446c0a53de0cb9fccc802b2a15e4c4cf03,0
0x000356313f1e2516da66f75321e9c3f64ed426c5bfe5731245cc64a29b7cfd09,0
0x0003fabe68f263c72e947448e12cf8438852975b6ae28c2a8a800ada202d4d3c,0
0x00042bf949c7a6ab8581e57856fd90f9144e9041999a4f5a3c4396c488870e9e,0
0x00047fa360175d3c1819a6093a242214b4299be14d58a0440b8046d62288b9e4,0


In [143]:
nawon3

,내원번호,환자번호,환자연령대,환자성별,환자거주지역,외래/응급여부,진료부서,DEPTT_NM,입원일자,퇴원일자
0,0x877cc6444a9b9f2b74e4ad59d5516ee270ceb1ecc16e...,0x000072b4c3266952b98b006c1ce118263c02a84febad...,80대,F,경기도 용인시,I,G3,G3,2020-05-26,2020-05-30
1,0x712909447c8aba949ab71cb1d31bcb200e2f20de77c9...,0x000072b4c3266952b98b006c1ce118263c02a84febad...,80대,F,경기도 용인시,E,EM,EM,2020-07-25,2020-07-25
2,0x9c8c89eefb40cf4de6a6ad5c45951ebd002d4b55354d...,0x00007445e281b90722ce72d2feabab3eb6ee61c4bef5...,30대,M,경기도 용인시,E,EM,EM,2020-09-25,2020-09-25
3,0xff8885d02154250e220f1c27d90b322931e831d89950...,0x00007445e281b90722ce72d2feabab3eb6ee61c4bef5...,30대,M,경기도 용인시,E,EM,EM,2020-02-02,2020-02-02
4,0x855a5dfe13c8617a96db7e8808635e1204e23c71bd6f...,0x00015b36e378adaf1e926dc33b9462fd691bd69a68fd...,30대,M,경기도 성남시,E,EM,EM,2020-04-14,2020-04-14
5,0x808b5149863595a910fa87246e836fc7e563383ac1b8...,0x00015b36e378adaf1e926dc33b9462fd691bd69a68fd...,30대,M,경기도 성남시,E,EM,EM,2021-05-21,2021-05-21
6,0x4c4568926eeb97fc2f1317812700e372cae303cf6c99...,0x00033515c693eed93f209060c7dbdf5ea9c9f95c6d87...,10대미만,F,경기도 시흥시,E,EM,EM,2020-10-12,2020-10-12
7,0xace171bb6510d3c9cd31ec1b34a5f61f373734c1f20b...,0x000336f2a0a12e71e63624e4b3c908446c0a53de0cb9...,50대,M,경기도 용인시,E,EM,OS,2020-08-11,2020-08-11
11,0x78bd28e6cc6544c71ee570faf2d6b1938c98d7d5bd18...,0x0003fabe68f263c72e947448e12cf8438852975b6ae2...,50대,M,경기도 용인시,I,M3,M3,2020-10-28,2020-10-28
